# **Knowledge Base**

### **Import delle dipendenze**
          - Importazione delle librerie utilizzate

In [1]:
#Importazione delle librerie
import pandas as pd
import random
import networkx as nx
import pickle

from pyswip import Prolog
from pyvis.network import Network
from typing import List, Dict
from collections import deque



### **Grafo KB**
        - Creazione del grafo partendo dai dati presenti nel dataset

In [4]:
# Carica il file CSV
csv_file = '../data/processed/Preprocessed_dataset_completo.csv'
df = pd.read_csv(csv_file)

# Crea un grafo diretto
G = nx.DiGraph()

# Itera sulle righe del DataFrame per costruire il grafo
for _, row in df.iterrows():
    game = row['name']
    developer = row['developer']
    genre = row['genery']
    platform = row['platform']
    publisher = row['publisher']

    # Aggiungi il gioco come nodo con tutti gli attributi
    G.add_node(game, type='game', developer=developer, user_score=row['user_score'], critics=row['critics'],
               users=row['users'], metacritic=row['metacritic'], rating=row['rating'], rating_top=row['rating_top'],
               playtime=row['playtime'], achievements_count=row['achievements_count'], ratings_count=row['ratings_count'],
               suggestions_count=row['suggestions_count'], game_series_count=row['game_series_count'],
               reviews_count=row['reviews_count'], added_status_yet=row['added_status_yet'],
               added_status_owned=row['added_status_owned'], added_status_beaten=row['added_status_beaten'],
               added_status_toplay=row['added_status_toplay'], added_status_dropped=row['added_status_dropped'],
               added_status_playing=row['added_status_playing'], released_year=row['released_year'],
               released_month=row['released_month'], released_day=row['released_day'], updated_year=row['updated_year'],
               updated_month=row['updated_month'], updated_day=row['updated_day'],
               is_in_top_100_publisher=row['is_in_top_100_publisher'],
               is_in_top_100_developer=row['is_in_top_100_developer'], genre=genre, platform=platform, publisher=publisher)

    # Aggiungi lo sviluppatore come nodo
    G.add_node(developer, type='developer')

    # Aggiungi un arco tra il gioco e lo sviluppatore
    G.add_edge(game, developer, relationship='developed_by')

print("Grafo costruito con successo.")

Grafo costruito con successo.


### **Visualizzazione del grafo interattivo**

        - Gafro parziale interattivo di 100 nodi

In [ ]:
# Carica il grafo dal file Pickle
with open("../kb/graph.pickle", "rb") as f:
    G_loaded = pickle.load(f)
print("Grafo caricato con successo.")

# Lista dei valori di num_start_nodes
num_start_nodes_list = [50, 100, 150]

# Itera su ogni valore di num_start_nodes
for num_start_nodes in num_start_nodes_list:
    # Crea una rete Pyvis
    net = Network(notebook=True, height="750px", width="100%", cdn_resources='remote')

    # Seleziona i nodi principali (es. giochi)
    start_nodes = [node for node in G_loaded.nodes if G_loaded.nodes[node].get('type') == 'game'][:num_start_nodes]

    # Usa BFS per selezionare nodi connessi
    sampled_nodes = set(start_nodes)
    queue = deque(start_nodes)

    while queue:
        node = queue.popleft()
        for neighbor in G_loaded.neighbors(node):
            if neighbor not in sampled_nodes:
                sampled_nodes.add(neighbor)
                queue.append(neighbor)

    # Filtra i nodi isolati (grado = 0)
    sampled_nodes = {node for node in sampled_nodes if G_loaded.degree(node) > 0}

    # Filtra gli archi che connettono i nodi selezionati
    sampled_edges = [edge for edge in G_loaded.edges if edge[0] in sampled_nodes and edge[1] in sampled_nodes]

    print(f"Numero di nodi selezionati per {num_start_nodes}:", len(sampled_nodes))
    print(f"Numero di archi selezionati per {num_start_nodes}:", len(sampled_edges))

    # Aggiungi nodi e archi filtrati alla rete
    for node in sampled_nodes:
        # Gestisci il campo 'title' per evitare errori
        title = str(node)  # Converti il nodo in stringa
        group = G_loaded.nodes[node].get('type', 'default')  # Usa 'default' se 'type' non è presente
        net.add_node(node, title=title, group=group)

    for edge in sampled_edges:
        # Gestisci il campo 'title' per gli archi
        edge_title = G_loaded.edges[edge].get('relationship', '')  # Usa una stringa vuota se 'relationship' non è presente
        net.add_edge(edge[0], edge[1], title=edge_title)

    # Salva e apri il grafo
    filename = f"../kb/grafo_{num_start_nodes}.html"
    net.show(filename)
    print(f"Grafo salvato in {filename}")

Grafo caricato con successo.
Numero di nodi selezionati per 50: 118
Numero di archi selezionati per 50: 191
../kb/grafo_50.html
Grafo salvato in ../kb/grafo_50.html
Numero di nodi selezionati per 100: 202
Numero di archi selezionati per 100: 377
../kb/grafo_100.html
Grafo salvato in ../kb/grafo_100.html
Numero di nodi selezionati per 150: 286
Numero di archi selezionati per 150: 570
../kb/grafo_150.html
Grafo salvato in ../kb/grafo_150.html


### **Prolog**


#### **Creazione del file Prolog**
        - Estrazione dei fatti e regole prolog dal grafo

In [10]:
def extract_prolog_facts(graph):
    prolog_facts = []
    prolog_rules = []

    for node, data in graph.nodes(data=True):
        if data.get('type') == 'game':
            # Estrai i campi del gioco
            game_name = node.replace("'", "''")  # Esegui l'escape degli apostrofi solo per il nome del gioco
            developer = data.get('developer', '').replace("'", "''")  # Esegui l'escape degli apostrofi per lo sviluppatore
            user_score = data.get('user_score', 0.0)
            metacritic = data.get('metacritic', 0.0)
            playtime = data.get('playtime', 0)
            achievements_count = data.get('achievements_count', 0)
            released_year = data.get('released_year', 0)
            released_month = data.get('released_month', 0)
            released_day = data.get('released_day', 0)
            is_in_top_100_publisher = data.get('is_in_top_100_publisher', 0)
            is_in_top_100_developer = data.get('is_in_top_100_developer', 0)
            genre = data.get('genre', '').replace("'", "''")  # Esegui l'escape degli apostrofi per il genere
            platform = data.get('platform', '').replace("'", "''")  # Esegui l'escape degli apostrofi per la piattaforma
            publisher = data.get('publisher', '').replace("'", "''")  # Esegui l'escape degli apostrofi per l'editore

            # Formatta il fatto Prolog
            prolog_fact = (
                f"game('{game_name}', '{developer}', {user_score}, {metacritic}, {playtime}, "
                f"{achievements_count}, {released_year}, {released_month}, {released_day}, "
                f"{is_in_top_100_publisher}, {is_in_top_100_developer}, '{genre}', '{platform}', '{publisher}')."
            )
            prolog_facts.append(prolog_fact)

    # Aggiungi le regole Prolog
    prolog_rules.append("""
% Regola per estrarre N elementi casuali da una lista
random_select(0, _, []).  % Caso base: se N è 0, restituisci una lista vuota
random_select(_, [], []). % Caso base: se la lista è vuota, restituisci una lista vuota
random_select(N, List, [Selected|Rest]) :-
    N > 0,
    length(List, Len),
    random_between(1, Len, Index),  % Genera un indice casuale
    nth1(Index, List, Selected),     % Seleziona elemento all'indice casuale
    delete(List, Selected, NewList), % Rimuovi elemento selezionato dalla lista
    N1 is N - 1,
    random_select(N1, NewList, Rest). % Ricorsione per selezionare gli altri elementi

% Regola per trovare la piattaforma con più giochi con User_Score > 8.5
platform_with_most_games(MaxPlatforms) :-
    findall(Platform, 
        (game(_, _, UserScore, _, _, _, _, _, _, _, _, _, Platform, _), 
        UserScore > 8.5), 
        PlatformsList),
    msort(PlatformsList, SortedPlatforms),
    clumped(SortedPlatforms, CountedPlatforms),
    max_member(_-MaxCount, CountedPlatforms),
    include(=(_-MaxCount), CountedPlatforms, MaxPlatforms).

% Regola per trovare lo sviluppatore con più giochi con User_Score > 8.5
developer_with_most_games(MaxDevelopers) :-
    findall(Developer, 
        (game(_, Developer, UserScore, _, _, _, _, _, _, _, _, _, _, _), 
        UserScore > 8.5), 
        DevelopersList),
    msort(DevelopersList, SortedDevelopers),
    clumped(SortedDevelopers, CountedDevelopers),
    max_member(_-MaxCount, CountedDevelopers),
    include(=(_-MaxCount), CountedDevelopers, MaxDevelopers).

% Regola per trovare i generi più popolari
most_popular_genres(MaxGenres) :-
    findall(Genre, 
        (game(_, _, UserScore, _, _, _, _, _, _, _, _, Genre, _, _), 
        UserScore > 8.5), 
        GenresList),
    msort(GenresList, SortedGenres),
    clumped(SortedGenres, CountedGenres),
    max_member(_-MaxCount, CountedGenres),
    include(=(_-MaxCount), CountedGenres, MaxGenres).

% Regola per trovare i giochi con il punteggio più alto degli utenti
top_rated_games(TopGames) :-
    findall(Game-UserScore, 
        (game(Game, _, UserScore, _, _, _, _, _, _, _, _, _, _, _), 
        UserScore > 8.5), 
        GamesList),
    keysort(GamesList, SortedGames),
    reverse(SortedGames, ReversedGames),
    random_select(5, ReversedGames, TopGames).  % Prendi i 5 giochi con il punteggio più alto

% Regola per trovare i giochi più giocati (playtime maggiore)
most_played_games(TopGames) :-
    findall(Game-Playtime, 
        (game(Game, _, _, _, Playtime, _, _, _, _, _, _, _, _, _), 
        Playtime > 50), 
        GamesList),
    keysort(GamesList, SortedGames),
    reverse(SortedGames, ReversedGames),
    random_select(5, ReversedGames, TopGames).  % Prendi i 5 giochi più giocati

% Regola per trovare i giochi con il maggior numero di achievement
most_achievements_games(TopGames) :-
    findall(Game-Achievements, 
        (game(Game, _, _, _, _, Achievements, _, _, _, _, _, _, _, _), 
        Achievements > 50), 
        GamesList),
    keysort(GamesList, SortedGames),
    reverse(SortedGames, ReversedGames),
    random_select(5, ReversedGames, TopGames).  % Prendi i 5 giochi con più achievement

% Regola per trovare i giochi più recenti (anno di rilascio)
most_recent_games(TopGames) :-
    findall(Game-ReleasedYear, 
        (game(Game, _, _, _, _, _, ReleasedYear, _, _, _, _, _, _, _), 
        ReleasedYear >= 2020), 
        GamesList),
    keysort(GamesList, SortedGames),
    reverse(SortedGames, ReversedGames),
    random_select(5, ReversedGames, TopGames).  % Prendi i 5 giochi più recenti
    """)
    return prolog_facts, prolog_rules

# Esempio di utilizzo
prolog_facts, prolog_rules = extract_prolog_facts(G)

# Salva i fatti e le regole in un file Prolog
def save_file_prolog(filepath, prolog_facts, prolog_rules):
    try:
        with open(filepath, "w", encoding="utf-8") as f:
            for fact in prolog_facts:
                f.write(fact + "\n")
            for rule in prolog_rules:
                f.write(rule + "\n")
        print(f"File Prolog '{filepath}' generato con successo!")
    except Exception as e:
        print(f"Errore durante il salvataggio del file: {e}")

# Salva i file Prolog
save_file_prolog("../kb/knowledge_base.pl", prolog_facts, prolog_rules)
save_file_prolog("../knowledge_base.pl", prolog_facts, prolog_rules)

File Prolog '../kb/knowledge_base.pl' generato con successo!
File Prolog '../knowledge_base.pl' generato con successo!


#### *Esempio di utilizzo della kb in Prolog*
        - Filtri utilizzati:    1: Categoria  2: Anno  
        - Valori dei filtri:    1: Action     2: 2008  

In [14]:
# Inizializza Prolog
prolog = Prolog()
try:
    prolog.consult('knowledge_base.pl')
    print("File consultato con successo.")
except Exception as e:
    print(f"Errore durante il caricamento: {e}")

# Mostra un menu di selezione all'utente e raccoglie i filtri scelti
def get_user_filters() -> List[str]:
    """Mostra il menu di selezione e ottiene i filtri scelti dall'utente."""
    print("\nPuoi filtrare i giochi aggiungendo uno o più dei seguenti criteri:")
    print("1. Genere (Genre)")
    print("2. Anno di uscita (Year)")
    print("3. Mese di uscita (Month)")
    print("4. Giorno di uscita (Day)")
    print("5. Sviluppatore (Developer)")
    print("6. Piattaforma (Platform)")
    print("7. Modifica il valore minimo di User_Score (default: > 8.5)")
    print("8. Trova la piattaforma con più giochi con User_Score > 8.5")
    print("9. Trova lo sviluppatore con più giochi con User_Score > 8.5")
    print("Premi invio senza inserire nulla per utilizzare solo il filtro di base (User_Score > 8.5).")

    user_filters = input("Scegli le caratteristiche da filtrare (inserisci i numeri corrispondenti separati da virgola): ").strip().split(',')
    return [f.strip() for f in user_filters]

# Applica i filtri scelti dall'utente alla query di base, modificandola in base alle scelte dell'utente.
def apply_filters(base_query: str, user_filters: List[str]) -> str:
    """Applica i filtri scelti dall'utente alla query di base."""
    for user_filter in user_filters:
        if user_filter == "1":
            genre = input("Inserisci il genere (es. Action Adventure, Sports): ").strip()
            base_query += f", Genre = '{genre}'"
        elif user_filter == "2":
            year = input("Inserisci l'anno di uscita (es. 2008, 2010): ").strip()
            if year.isdigit():
                base_query += f", Released_Year = {year}"
            else:
                print("Anno non valido. Verrà ignorato.")
        elif user_filter == "3":
            month = input("Inserisci il mese di uscita (1-12): ").strip()
            if month.isdigit() and 1 <= int(month) <= 12:
                base_query += f", Released_Month = {month}"
            else:
                print("Mese non valido. Verrà ignorato.")
        elif user_filter == "4":
            day = input("Inserisci il giorno di uscita (1-31): ").strip()
            if day.isdigit() and 1 <= int(day) <= 31:
                base_query += f", Released_Day = {day}"
            else:
                print("Giorno non valido. Verrà ignorato.")
        elif user_filter == "5":
            developer = input("Inserisci lo sviluppatore (es. Nintendo, Sony): ").strip()
            base_query += f", Developer = '{developer}'"
        elif user_filter == "6":
            platform = input("Inserisci la piattaforma (es. PC, PlayStation, Xbox): ").strip()
            base_query += f", Platform = '{platform}'"
        elif user_filter == "7":
            user_score = input("Inserisci il valore minimo di User_Score (es. 9.0, 9.5): ").strip()
            if user_score.replace('.', '', 1).isdigit():
                base_query = base_query.replace("User_Score > 8.5", f"User_Score > {user_score}")
            else:
                print("Valore di User_Score non valido. Verrà mantenuto il valore di default (> 8.5).")
    return base_query

# Esegue la query su Prolog e restituisce i risultati.
def execute_query(base_query: str) -> List[Dict]:
    """Esegue la query su Prolog e restituisce i risultati."""
    print("\nEseguo la query...")
    try:
        results = list(prolog.query(base_query))
    except Exception as e:
        print(f"Errore durante l'esecuzione della query: {e}")
        return []
    return results

# Mostra i risultati della query all'utente, limitandosi a 10 risultati casuali.
def display_results(results: List[Dict]):
    """Mostra i risultati della query."""
    if not results:
        print("Nessun risultato trovato con i filtri specificati.")
        return

    # Seleziona fino a 10 risultati casuali
    random_results = random.sample(results, min(len(results), 10))

    # Mostra i risultati
    print("\nRisultati della query (primi 10 casuali con User_Score > 8.5 e i filtri scelti):")
    for result in random_results:
        print(f"Name: {result['Name']},\n "
              f"- User Score: {result['User_Score']},\n"
              f"- Developer: {result['Developer']},\n"
              f"- Released Year/Month/Day : {result['Released_Year']}-{result['Released_Month']}-{result['Released_Day']}\n"
              f"- Genre: {result['Genre']},\n"
              f"- Platform: {result['Platform']}")

# Funzione principale: Coordina l'intero processo di interrogazione della base di conoscenza
def query_games():
    """Funzione principale per interrogare la base di conoscenza."""
    base_query = (
    "game(Name, User_Score, Critics, Users, Metacritic, Rating, Rating_Top, Playtime, "
    "Achievements_Count, Ratings_Count, Suggestions_Count, Game_Series_Count, Reviews_Count, "
    "Added_Status_Yet, Added_Status_Owned, Added_Status_Beaten, Added_Status_Toplay, "
    "Added_Status_Dropped, Added_Status_Playing, Released_Year, Released_Month, Released_Day, "
    "Updated_Year, Updated_Month, Updated_Day, Is_In_Top_100_Publisher, Is_In_Top_100_Developer, "
    "Genre, Platform, Publisher, Developer), User_Score > 8.5"
)

    # Ottieni i filtri scelti dall'utente
    user_filters = get_user_filters()


    # Applica i filtri alla query di base
    base_query = apply_filters(base_query, user_filters)
    # Esegui la query
    results = execute_query(base_query)
    # Mostra i risultati
    display_results(results)

# Esempio interattivo
print("Benvenuto nella base di conoscenza dei videogiochi!")
query_games()

File consultato con successo.
Benvenuto nella base di conoscenza dei videogiochi!

Puoi filtrare i giochi aggiungendo uno o più dei seguenti criteri:
1. Genere (Genre)
2. Anno di uscita (Year)
3. Mese di uscita (Month)
4. Giorno di uscita (Day)
5. Sviluppatore (Developer)
6. Piattaforma (Platform)
7. Modifica il valore minimo di User_Score (default: > 8.5)
8. Trova la piattaforma con più giochi con User_Score > 8.5
9. Trova lo sviluppatore con più giochi con User_Score > 8.5
Premi invio senza inserire nulla per utilizzare solo il filtro di base (User_Score > 8.5).

Eseguo la query...

Risultati della query (primi 10 casuali con User_Score > 8.5 e i filtri scelti):
Name: Super Smash Bros. Brawl,
 - User Score: 8.8,
- Developer: Intelligent Systems,
- Released Year/Month/Day : 2008-3-9
- Genre: Action,
- Platform: Wii
Name: Castle Crashers,
 - User Score: 8.8,
- Developer: The Behemoth,
- Released Year/Month/Day : 2008-8-27
- Genre: Action,
- Platform: PC
Name: Left 4 Dead,
 - User Score